In [2]:
D.<x> =PolynomialRing(QQ)
DD=PolynomialRing(QQ,'q')
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

def Delta(n):
    M = ModularForms(SL2Z, 12) 
    return DD(M.q_expansion_basis(n)[0])

def tau(n):
    return Delta(n+1)[n]
 
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})


print("okay")

okay


In [70]:
ls=[]
for n in [1..5]:ls=ls+[tau(n)]
print(ls)

[1, -24, 252, -1472, 4830]


In [71]:
def RamanujanCongruenceSeries(n): 
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)

print(RamanujanCongruenceSeries(5))



-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [3]:
def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)


def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+k)
    return g[0]

for k in [1..3]:
    print(RamanujanCongruenceSeries(k));print()
    print(recip(k)^k);print()
    rpc=recipPowerConstantTerm(k)
    test=order(2,rpc)-3*digitsum(2,k)
    print((rpc,test))
    
    print("----------------------------------------------------------")

-24*x^2 + x

x^-1 + 24 + 576*x + O(x^2)

(24, 0)
----------------------------------------------------------
252*x^3 - 24*x^2 + x

x^-2 + 48*x^-1 + 1224 + 19008*x + O(x^2)

(1224, 0)
----------------------------------------------------------
-1472*x^4 + 252*x^3 - 24*x^2 + x

x^-3 + 72*x^-2 + 2700*x^-1 + 70080 + 1427040*x + O(x^2)

(70080, 0)
----------------------------------------------------------


In [4]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..10]: # 100 < precision is necessary
        start=time.time()
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        ds=digitsum(3,a)
        test=o3-ds
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.051553964614868164)
(1, 2, 0, 0.04049181938171387)
(1, 3, 0, 0.06464099884033203)
(1, 4, 0, 0.08203697204589844)
(1, 5, 0, 0.08393716812133789)
(1, 6, 0, 0.05292487144470215)
(1, 7, 0, 0.0919790267944336)
(1, 8, 0, 0.09465479850769043)
(1, 9, 0, 0.10274219512939453)
(1, 10, 0, 0.06233787536621094)

bad a's for trial  1

[]
------------------------------------------------------------------
no:

[(1, 0)]


In [6]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+k)
   
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..100]: # 100 < precision is necessary
        
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        ds=digitsum(3,a)
        test=o3-ds
        tests=tests+[test]
        print((b,a,test))
    print("trial",b)
    print(tests)
    print("-----------------------------------------------------------------------")


(1, 1, 0)
(1, 2, 0)
(1, 3, 0)
(1, 4, 0)
(1, 5, 0)
(1, 6, 0)
(1, 7, 0)
(1, 8, 0)
(1, 9, 0)
(1, 10, 0)
(1, 11, 0)
(1, 12, 0)
(1, 13, 0)
(1, 14, 0)
(1, 15, 0)
(1, 16, 0)
(1, 17, 0)
(1, 18, 0)
(1, 19, 0)
(1, 20, 0)
(1, 21, 0)
(1, 22, 0)
(1, 23, 0)
(1, 24, 0)
(1, 25, 0)
(1, 26, 0)
(1, 27, 0)
(1, 28, 0)
(1, 29, 0)
(1, 30, 0)
(1, 31, 0)
(1, 32, 0)
(1, 33, 0)
(1, 34, 0)
(1, 35, 0)
(1, 36, 0)
(1, 37, 0)
(1, 38, 0)
(1, 39, 0)
(1, 40, 0)
(1, 41, 0)
(1, 42, 0)
(1, 43, 0)
(1, 44, 0)
(1, 45, 0)
(1, 46, 0)
(1, 47, 0)
(1, 48, 0)
(1, 49, 0)
(1, 50, 0)
(1, 51, 0)
(1, 52, 0)
(1, 53, 0)
(1, 54, 0)
(1, 55, 0)
(1, 56, 0)
(1, 57, 0)
(1, 58, 0)
(1, 59, 0)
(1, 60, 0)
(1, 61, 0)
(1, 62, 0)
(1, 63, 0)
(1, 64, 0)
(1, 65, 0)
(1, 66, 0)
(1, 67, 0)
(1, 68, 0)
(1, 69, 0)
(1, 70, 0)
(1, 71, 0)
(1, 72, 0)
(1, 73, 0)
(1, 74, 0)
(1, 75, 0)
(1, 76, 0)
(1, 77, 0)
(1, 78, 0)
(1, 79, 0)
(1, 80, 0)
(1, 81, 0)
(1, 82, 0)
(1, 83, 0)
(1, 84, 0)
(1, 85, 0)
(1, 86, 0)
(1, 87, 0)
(1, 88, 0)
(1, 89, 0)
(1, 90, 0)
(1, 91, 0)
(1, 92,

In [7]:
import random
import time


def specialModulus(a):
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
tests=[]
for b in [1..3]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        sk=c/3^o3
        m=mod(sk,3)
        if ((mod(a,2)==1)&(m!=2)):no=no+[a]
        if ((mod(a,2)==0)&(m!=1)):no=no+[a]
        ds=digitsum(3,a)
        test=o3-ds
        tests=tests+[test]
        print((b,a,m,len(no),test))
    print();print("trial",b)
    print();print(tests)
    print("-----------------------------------------------------------------------")


(1, 1, 2, 0, 0)
(1, 2, 1, 0, 0)
(1, 3, 2, 0, 0)
(1, 4, 1, 0, 0)
(1, 5, 2, 0, 0)
(1, 6, 1, 0, 0)
(1, 7, 2, 0, 0)
(1, 8, 1, 0, 0)
(1, 9, 2, 0, 0)
(1, 10, 1, 0, 0)
(1, 11, 2, 0, 0)
(1, 12, 1, 0, 0)
(1, 13, 2, 0, 0)
(1, 14, 1, 0, 0)
(1, 15, 2, 0, 0)
(1, 16, 1, 0, 0)
(1, 17, 2, 0, 0)
(1, 18, 1, 0, 0)
(1, 19, 2, 0, 0)
(1, 20, 1, 0, 0)
(1, 21, 2, 0, 0)
(1, 22, 1, 0, 0)
(1, 23, 2, 0, 0)
(1, 24, 1, 0, 0)
(1, 25, 2, 0, 0)
(1, 26, 1, 0, 0)
(1, 27, 2, 0, 0)
(1, 28, 1, 0, 0)
(1, 29, 2, 0, 0)
(1, 30, 1, 0, 0)
(1, 31, 2, 0, 0)
(1, 32, 1, 0, 0)
(1, 33, 2, 0, 0)
(1, 34, 1, 0, 0)
(1, 35, 2, 0, 0)
(1, 36, 1, 0, 0)
(1, 37, 2, 0, 0)
(1, 38, 1, 0, 0)
(1, 39, 2, 0, 0)
(1, 40, 1, 0, 0)
(1, 41, 2, 0, 0)
(1, 42, 1, 0, 0)
(1, 43, 2, 0, 0)
(1, 44, 1, 0, 0)
(1, 45, 2, 0, 0)
(1, 46, 1, 0, 0)
(1, 47, 2, 0, 0)
(1, 48, 1, 0, 0)
(1, 49, 2, 0, 0)
(1, 50, 1, 0, 0)
(1, 51, 2, 0, 0)
(1, 52, 1, 0, 0)
(1, 53, 2, 0, 0)
(1, 54, 1, 0, 0)
(1, 55, 2, 0, 0)
(1, 56, 1, 0, 0)
(1, 57, 2, 0, 0)
(1, 58, 1, 0, 0)
(1, 59, 2, 0, 0)
(1, 6

(1, 463, 2, 0, 0)
(1, 464, 1, 0, 0)
(1, 465, 2, 0, 0)
(1, 466, 1, 0, 0)
(1, 467, 2, 0, 0)
(1, 468, 1, 0, 0)
(1, 469, 2, 0, 0)
(1, 470, 1, 0, 0)
(1, 471, 2, 0, 0)
(1, 472, 1, 0, 0)
(1, 473, 2, 0, 0)
(1, 474, 1, 0, 0)
(1, 475, 2, 0, 0)
(1, 476, 1, 0, 0)
(1, 477, 2, 0, 0)
(1, 478, 1, 0, 0)
(1, 479, 2, 0, 0)
(1, 480, 1, 0, 0)
(1, 481, 2, 0, 0)
(1, 482, 1, 0, 0)
(1, 483, 2, 0, 0)
(1, 484, 1, 0, 0)
(1, 485, 2, 0, 0)
(1, 486, 1, 0, 0)
(1, 487, 2, 0, 0)
(1, 488, 1, 0, 0)
(1, 489, 2, 0, 0)
(1, 490, 1, 0, 0)
(1, 491, 2, 0, 0)
(1, 492, 1, 0, 0)
(1, 493, 2, 0, 0)
(1, 494, 1, 0, 0)
(1, 495, 2, 0, 0)
(1, 496, 1, 0, 0)
(1, 497, 2, 0, 0)
(1, 498, 1, 0, 0)
(1, 499, 2, 0, 0)
(1, 500, 1, 0, 0)

trial 1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(2, 337, 2, 0, 0)
(2, 338, 1, 0, 0)
(2, 339, 2, 0, 0)
(2, 340, 1, 0, 0)
(2, 341, 2, 0, 0)
(2, 342, 1, 0, 0)
(2, 343, 2, 0, 0)
(2, 344, 1, 0, 0)
(2, 345, 2, 0, 0)
(2, 346, 1, 0, 0)
(2, 347, 2, 0, 0)
(2, 348, 1, 0, 0)
(2, 349, 2, 0, 0)
(2, 350, 1, 0, 0)
(2, 351, 2, 0, 0)
(2, 352, 1, 0, 0)
(2, 353, 2, 0, 0)
(2, 354, 1, 0, 0)
(2, 355, 2, 0, 0)
(2, 356, 1, 0, 0)
(2, 357, 2, 0, 0)
(2, 358, 1, 0, 0)
(2, 359, 2, 0, 0)
(2, 360, 1, 0, 0)
(2, 361, 2, 0, 0)
(2, 362, 1, 0, 0)
(2, 363, 2, 0, 0)
(2, 364, 1, 0, 0)
(2, 365, 2, 0, 0)
(2, 366, 1, 0, 0)
(2, 367, 2, 0, 0)
(2, 368, 1, 0, 0)
(2, 369, 2, 0, 0)
(2, 370, 1, 0, 0)
(2, 371, 2, 0, 0)
(2, 372, 1, 0, 0)
(2, 373, 2, 0, 0)
(2, 374, 1, 0, 0)
(2, 375, 2, 0, 0)
(2, 376, 1, 0, 0)
(2, 377, 2, 0, 0)
(2, 378, 1, 0, 0)
(2, 379, 2, 0, 0)
(2, 380, 1, 0, 0)
(2, 381, 2, 0, 0)
(2, 382, 1, 0, 0)
(2, 383, 2, 0, 0)
(2, 384, 1, 0, 0)
(2, 385, 2, 0, 0)
(2, 386, 1, 0, 0)
(2, 387, 2, 0, 0)
(2, 388, 1, 0, 0)
(2, 389, 2, 0, 0)
(2, 390, 1, 0, 0)
(2, 391, 2, 0, 0)
(2, 392, 1

(3, 127, 2, 0, 0)
(3, 128, 1, 0, 0)
(3, 129, 2, 0, 0)
(3, 130, 1, 0, 0)
(3, 131, 2, 0, 0)
(3, 132, 1, 0, 0)
(3, 133, 2, 0, 0)
(3, 134, 1, 0, 0)
(3, 135, 2, 0, 0)
(3, 136, 1, 0, 0)
(3, 137, 2, 0, 0)
(3, 138, 1, 0, 0)
(3, 139, 2, 0, 0)
(3, 140, 1, 0, 0)
(3, 141, 2, 0, 0)
(3, 142, 1, 0, 0)
(3, 143, 2, 0, 0)
(3, 144, 1, 0, 0)
(3, 145, 2, 0, 0)
(3, 146, 1, 0, 0)
(3, 147, 2, 0, 0)
(3, 148, 1, 0, 0)
(3, 149, 2, 0, 0)
(3, 150, 1, 0, 0)
(3, 151, 2, 0, 0)
(3, 152, 1, 0, 0)
(3, 153, 2, 0, 0)
(3, 154, 1, 0, 0)
(3, 155, 2, 0, 0)
(3, 156, 1, 0, 0)
(3, 157, 2, 0, 0)
(3, 158, 1, 0, 0)
(3, 159, 2, 0, 0)
(3, 160, 1, 0, 0)
(3, 161, 2, 0, 0)
(3, 162, 1, 0, 0)
(3, 163, 2, 0, 0)
(3, 164, 1, 0, 0)
(3, 165, 2, 0, 0)
(3, 166, 1, 0, 0)
(3, 167, 2, 0, 0)
(3, 168, 1, 0, 0)
(3, 169, 2, 0, 0)
(3, 170, 1, 0, 0)
(3, 171, 2, 0, 0)
(3, 172, 1, 0, 0)
(3, 173, 2, 0, 0)
(3, 174, 1, 0, 0)
(3, 175, 2, 0, 0)
(3, 176, 1, 0, 0)
(3, 177, 2, 0, 0)
(3, 178, 1, 0, 0)
(3, 179, 2, 0, 0)
(3, 180, 1, 0, 0)
(3, 181, 2, 0, 0)
(3, 182, 1